In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\train.parquet')
oos = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oos.parquet')
oot = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oot.parquet')

In [7]:
df.columns

Index(['C_GIOITINH', 'TTHONNHAN', 'NHANVIENBIDV', 'BASE_AUM', 'TUOI', 'INCOME',
       'CBAL', 'AFLIMT_AVG', 'LTV', 'N_AVG_DEPOSIT_12M', 'FLAG_SALARY_ACC',
       'FLAG_DEPOSIT', 'UTILIZATION_RATE', 'CNT_CREDIT_CARDS',
       'AMT_CASH_ADVANCE_12M', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M',
       'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 'MOB', 'CNT_OTHER_PRODUCTS',
       'LIMIT_TO_INCOME', 'AMT_VAR_6M', 'CBAL_SHORTTERM_LOAN',
       'CBAL_LONGTERM_LOAN', 'HAS_LONGTERM_LOAN', 'CNT_DPD_30PLUS_6M',
       'OCCUPATION_TYPE', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'MAX_DPD_12M',
       'AVG_OD_DPD_12M', 'MAX_NHOMNOCIC', 'N_AVG_OVERDUE_CBAL_12M',
       'BAD_NEXT_12M'],
      dtype='object')

In [8]:
df.shape 

(1137807, 38)

In [3]:
# Danh sách các biến cần ép kiểu về Category (Nominal & Binary)
categorical_cols = [
    'OCCUPATION_TYPE', 
    'PURCOD_MAX', 
    'PURCOD_MIN',
    'SOHUUNHA', 
    'NHANVIENBIDV',
    'FLAG_SALARY_ACC', 
    'FLAG_DEPOSIT', 
    'FLAG_CASH_ADVANCE',
    'HAS_SHORTTERM_LOAN', # Nếu có trong df final
    'HAS_LONGTERM_LOAN',  # Nếu có trong df final
    'BAD_CURRENT',        # Nếu giữ lại làm feature (thường là drop vì data leakage)
    'XULYNO'              # Nếu giữ lại
]

# Lọc những cột thực sự tồn tại trong df (đề phòng bạn đã drop bớt)
existing_cat_cols = [col for col in categorical_cols if col in df.columns]

# Chuyển đổi
for col in existing_cat_cols:
    df[col] = df[col].astype('str') # Chuyển về string để EBM hiểu là category
    oos[col] = oos[col].astype('str')
    oot[col] = oot[col].astype('str')

In [4]:
y_train = df['BAD_NEXT_12M']
y_oos = oos['BAD_NEXT_12M']
y_oot = oot['BAD_NEXT_12M']
X = df.drop('BAD_NEXT_12M', axis=1)
X_oos = oos.drop('BAD_NEXT_12M', axis=1)
X_oot = oot.drop('BAD_NEXT_12M', axis=1)

In [5]:
numeric_cols = X.select_dtypes(include=[np.number]).columns
corr_matrix = X[numeric_cols].corr().abs() # Lấy trị tuyệt đối (âm hay dương đều là tương quan mạnh)

# 2. Chọn ngưỡng cắt (Threshold)
# Với EBM, ngưỡng an toàn là 0.7. Ngưỡng 0.8 là bắt buộc phải xử lý.
threshold = 0.7

# 3. Lọc ra các cặp biến vượt ngưỡng
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Tạo danh sách chi tiết để bạn dễ quyết định giữ ai/bỏ ai
high_corr_pairs = []
for col in upper.columns:
    high_cols = upper.index[upper[col] > threshold].tolist()
    for row in high_cols:
        val = upper.loc[row, col]
        high_corr_pairs.append((row, col, val))

# Sắp xếp theo độ tương quan giảm dần
high_corr_pairs.sort(key=lambda x: x[2], reverse=True)

# 4. In kết quả
print(f"Tìm thấy {len(high_corr_pairs)} cặp biến tương quan cao (> {threshold}):")
print("-" * 60)
print(f"{'Biến 1 (Giữ?)':<30} | {'Biến 2 (Bỏ?)':<30} | {'Corr':<10}")
print("-" * 60)
for pair in high_corr_pairs:
    print(f"{pair[0]:<30} | {pair[1]:<30} | {pair[2]:.4f}")

Tìm thấy 9 cặp biến tương quan cao (> 0.7):
------------------------------------------------------------
Biến 1 (Giữ?)                  | Biến 2 (Bỏ?)                   | Corr      
------------------------------------------------------------
INCOME                         | AFLIMT_AVG                     | 0.8976
UTILIZATION_RATE               | DTI_RATIO                      | 0.8694
BASE_AUM                       | N_AVG_DEPOSIT_12M              | 0.7862
DURATION_MAX                   | REMAINING_DURATION_MAX         | 0.7833
CBAL                           | AFLIMT_AVG                     | 0.7658
DURATION_MAX                   | TIME_TO_OP_MAX                 | 0.7511
MAX_DPD_12M                    | MAX_NHOMNOCIC                  | 0.7483
CBAL                           | N_AVG_OVERDUE_CBAL_12M         | 0.7232
CBAL                           | DTI_RATIO                      | 0.7229


Trong các mô hình dựa trên cây (Tree-based) như EBM, XGBoost, LightGBM, ngưỡng xử lý đa cộng tuyến thường lỏng hơn nhiều so với Logistic Regression. Ngưỡng vàng: Thường là 0.9 hoặc 0.95. Nếu tương quan < 0.9, mô hình vẫn đủ sức phân biệt được tín hiệu riêng biệt của từng biến.

In [6]:
X.to_parquet('X_train.parquet', index=False)
X_oos.to_parquet('X_oos.parquet', index=False)
X_oot.to_parquet('X_oot.parquet', index=False)
y_train.to_frame().to_parquet('y_train.parquet', index=False)
y_oos.to_frame().to_parquet('y_oos.parquet', index=False)
y_oot.to_frame().to_parquet('y_oot.parquet', index=False)